In [24]:
import h2o
from h2o.automl import H2OAutoML

from sklearn.metrics import accuracy_score

In [61]:
h2o.init()
#df = h2o.import_file("../input/real-quad-fields-1-2/real_quad_fields_1_2.csv")
df = h2o.import_file("C:/Temp/real_quad_fields_1_2.csv")

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 hours 28 mins
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.5
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_from_python_esultano_czcmdl
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,6.601 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [62]:
print(df['class_number'].table())

class_number,Count
1,177159
2,183432


In [37]:
#df = df.split_frame(ratios=[0.25])[0]
#print(df['class_number'].table())

class_number,Count
1,228
2,249


In [63]:
# H2O uses cross validation internally during training (no validation_frame is needed)
train, test = df.split_frame(ratios=[0.3], seed=42)

# Split the data into Train/Validation/Test with Train having 30% and test and validation 35% each
#train, valid, test = df.split_frame(ratios=[.3, .35])

Columns are:
*  'defining_polynomial'
*  'class_number'
*  'discriminant'
*  'res_s=0'
*  'number_roots_unity'
*  'regulator'
*  'a_1', 'a_2', 'a_3', ..., 'a_1000'

In [64]:
primes_up_to_1000 = [1, 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997]
x = [i + 5 for i in primes_up_to_1000]
y = "class_number"

# for binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [40]:
# maybe include "GLM" "GBM" "DeepLearning"
autoML = H2OAutoML(max_runtime_secs=18000, include_algos=["DRF"], balance_classes=True)
autoML.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


See [this stack overflow post](https://stackoverflow.com/questions/55081358/how-to-find-best-params-of-leader-model-in-automl-h2o-python) for details on finding best params of leader model

In [41]:
autoML.leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
XRT_1_AutoML_20210820_153437,0.454073,0.72229,0.481127,0.5,0.513848,0.26404
DRF_1_AutoML_20210820_153437,0.433625,0.743847,0.443974,0.493671,0.523293,0.273836


In [42]:
m = h2o.get_model("DRF_1_AutoML_20210820_153437")
print(m.params.keys())
m.params['nfolds']

dict_keys(['model_id', 'training_frame', 'validation_frame', 'nfolds', 'keep_cross_validation_models', 'keep_cross_validation_predictions', 'keep_cross_validation_fold_assignment', 'score_each_iteration', 'score_tree_interval', 'fold_assignment', 'fold_column', 'response_column', 'ignored_columns', 'ignore_const_cols', 'offset_column', 'weights_column', 'balance_classes', 'class_sampling_factors', 'max_after_balance_size', 'max_confusion_matrix_size', 'ntrees', 'max_depth', 'min_rows', 'nbins', 'nbins_top_level', 'nbins_cats', 'r2_stopping', 'stopping_rounds', 'stopping_metric', 'stopping_tolerance', 'max_runtime_secs', 'seed', 'build_tree_one_node', 'mtries', 'sample_rate', 'sample_rate_per_class', 'binomial_double_trees', 'checkpoint', 'col_sample_rate_change_per_level', 'col_sample_rate_per_tree', 'min_split_improvement', 'histogram_type', 'categorical_encoding', 'calibrate_model', 'calibration_frame', 'distribution', 'custom_metric_func', 'export_checkpoints_dir', 'check_constant_r

{'default': 0, 'actual': 5, 'input': 5}

In [65]:
y_pred = m.predict(test)
print("Test accuracy:", accuracy_score(h2o.as_list(test)[y].tolist(), h2o.as_list(y_pred)["predict"].tolist()))
print(m.model_performance(test).accuracy())
print(m.model_performance(test).F1())

drf prediction progress: |████████████████████████████████████████████████| 100%
Test accuracy: 0.5089843146115518
[[0.36254980079681276, 0.5105183670071708]]
[[0.16749256689791872, 0.6742237477725622]]
